# Прогнозирование стоимости автомобиля по характеристикам

# Подготовительные операции

In [1]:
import numpy as np
import pandas as pd
import sys
import re

from sklearn.model_selection import KFold,train_test_split
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor, VotingRegressor

from tqdm.notebook import tqdm

from catboost import CatBoostRegressor

import klib

import automl_alex
from automl_alex import AutoMLRegressor, BestSingleModelRegressor

from pandas_profiling import ProfileReport

pd.set_option('max_columns', None)

In [2]:
def mape(y_true, y_pred):
    return round(np.mean(np.abs((y_pred - y_true) / y_true)*100), 3)

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Подгружаем данные

In [4]:
train = pd.read_csv('input/autoru-parsed-0603-1304/new_data_99_06_03_13_04.csv')
test = pd.read_csv('input/sf-dst-car-price/test.csv')
sample_submission = pd.read_csv('input/sf-dst-car-price/sample_submission.csv')

# EDA

С помощью пакета Pandas Profiling легко осуществляем EDA анализ датасета и получаем все нужные нам характеристики в одном месте.

In [5]:
#profile = ProfileReport(train, title="Pandas Profiling Report")

In [6]:
#profile.to_notebook_iframe()

In [7]:
#profile.to_file("your_report.html")

# Data Preprocessing

In [8]:
train = train.loc[(train.brand == "BMW")]

In [9]:
klib.data_cleaning(train)

Shape of cleaned data: (14390, 20)Remaining NAs: 9438

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 2
     of which 2 single valued.     Columns: ['brand', 'таможня']
Dropped missing values: 0
Reduced memory by at least: 0.79 MB (-32.51%)


,unnamed_0,body_type,color,fuel_type,model_date,name,number_of_doors,production_date,vehicle_configuration,engine_displacement,engine_power,description,mileage,комплектация,привод,руль,владельцы,птс,владение,price
4760,4760,Хэтчбек 5 дв.,040001,бензин,2007.0,116i 1.6 AT (115 л.с.),5.0,2008,AUTOMATIC,116i,115.0,Вся в родне в оригинале продаю так как 3 машин...,115000,{'id': '0'},задний,LEFT,2.0,ORIGINAL,NaN,436000
4761,4761,Хэтчбек 5 дв.,040001,бензин,2007.0,116i 1.6 AT (115 л.с.),5.0,2011,AUTOMATIC,116i,115.0,Приобретался новым у официального дилера BMW А...,39728,"{'id': '2430196', 'name': '116', 'available_op...",задний,LEFT,1.0,ORIGINAL,"{'year': 2011, 'month': 4}",595000
4762,4762,Хэтчбек 5 дв.,FAFBFB,бензин,2017.0,118i 1.5 AT (136 л.с.),5.0,2017,AUTOMATIC,118i,136.0,Машина в отличном состоянии. Не курили. Все то...,41000,{'id': '0'},задний,LEFT,2.0,ORIGINAL,NaN,1150000
4763,4763,Хэтчбек 5 дв.,EE1D19,бензин,2011.0,116i 1.6 AT (136 л.с.),5.0,2014,AUTOMATIC,116i,136.0,МБ-Беляево — официальный дилер «Мерседес-Бенц»...,87284,"{'id': '7707468', 'name': '116i', 'available_o...",задний,LEFT,3.0,ORIGINAL,NaN,660000
4764,4764,Хэтчбек 5 дв.,FAFBFB,бензин,2011.0,116i 1.6 AT (136 л.с.),5.0,2013,AUTOMATIC,116i,136.0,"Машина в хорошем состоянии покупал у знакомых,...",124000,"{'id': '7707468', 'name': '116i', 'available_o...",задний,LEFT,3.0,ORIGINAL,NaN,575000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108321,108321,Внедорожник 5 дв.,0000CC,бензин,2012.0,35i 3.0 AT (306 л.с.) 4WD,5.0,2013,AUTOMATIC,35i,306.0,Автомобиль представлен официальным дилером BMW...,76670,{'id': '0'},полный,LEFT,3.0,ORIGINAL,NaN,1795000
108322,108322,Внедорожник 5 дв.,040001,дизель,2014.0,M50d 3.0d AT (381 л.с.) 4WD,5.0,2016,AUTOMATIC,M50d,381.0,Максимальная комплектация По любым вопросам зв...,101000,{'id': '0'},полный,LEFT,1.0,ORIGINAL,"{'year': 2016, 'month': 8}",3580000
108323,108323,Внедорожник 5 дв.,040001,дизель,2014.0,30d 3.0d AT (249 л.с.) 4WD,5.0,2016,AUTOMATIC,30d,249.0,<NA>,108000,"{'id': '21075602', 'name': 'xDrive30d', 'avail...",полный,LEFT,1.0,ORIGINAL,"{'year': 2016, 'month': 6}",2600000
108324,108324,Внедорожник 5 дв.,FAFBFB,бензин,2009.0,4.4 AT (555 л.с.) 4WD,5.0,2011,AUTOMATIC,4.4,555.0,КОМПЛЕКТАЦИЯ: INDIVIDUAL Я первый и единственн...,143400,"{'id': '4731958', 'name': 'X6 M', 'available_o...",полный,LEFT,1.0,ORIGINAL,"{'year': 2011, 'month': 9}",1450000


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14390 entries, 4760 to 108325
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            14390 non-null  int64  
 1   bodyType              14390 non-null  object 
 2   brand                 14390 non-null  object 
 3   color                 14390 non-null  object 
 4   fuelType              14390 non-null  object 
 5   modelDate             14390 non-null  float64
 6   name                  14390 non-null  object 
 7   numberOfDoors         14390 non-null  float64
 8   productionDate        14390 non-null  int64  
 9   vehicleConfiguration  14390 non-null  object 
 10  engineDisplacement    14390 non-null  object 
 11  enginePower           14390 non-null  float64
 12  description           14081 non-null  object 
 13  mileage               14390 non-null  int64  
 14  Комплектация          14390 non-null  object 
 15  Привод         

In [11]:
train.sample(3)

,Unnamed: 0,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Владельцы,ПТС,Таможня,Владение,Price
85528,85528,Внедорожник 5 дв.,BMW,040001,дизель,2014.0,30d 3.0d AT (249 л.с.) 4WD,5.0,2017,AUTOMATIC,30d,249.0,Куплена в 2018 году в мае \nВ идеальном состоя...,49000,"{'id': '21041261', 'name': 'xDrive30d', 'avail...",полный,LEFT,1.0,ORIGINAL,True,"{'year': 2018, 'month': 5}",2820000
79728,79728,Купе,BMW,040001,бензин,2005.0,325xi 2.5 AT (218 л.с.) 4WD,2.0,2007,AUTOMATIC,325xi,218.0,"автомобиль в отличном техническом состоянии, е...",152000,"{'id': '4721835', 'name': '325', 'available_op...",полный,LEFT,3.0,ORIGINAL,True,NaN,640000
8512,8512,Купе,BMW,FAFBFB,бензин,2011.0,640i xDrive 3.0 AT (320 л.с.) 4WD,2.0,2013,AUTOMATIC,640i,320.0,РОЛЬФ Премиум – крупнейший официальный дилер М...,94874,{'id': '0'},полный,LEFT,2.0,ORIGINAL,True,NaN,1795000


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837 entries, 0 to 3836
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              3837 non-null   object 
 1   brand                 3837 non-null   object 
 2   color                 3837 non-null   object 
 3   fuelType              3837 non-null   object 
 4   modelDate             3837 non-null   float64
 5   name                  3837 non-null   object 
 6   numberOfDoors         3837 non-null   float64
 7   productionDate        3837 non-null   float64
 8   vehicleConfiguration  3837 non-null   object 
 9   vehicleTransmission   3837 non-null   object 
 10  engineDisplacement    3837 non-null   object 
 11  enginePower           3837 non-null   object 
 12  description           3837 non-null   object 
 13  mileage               3837 non-null   float64
 14  Комплектация          3837 non-null   object 
 15  Привод               

In [13]:
test.sample(3)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
1315,универсал 5 дв.,BMW,серебристый,дизель,2005.0,320d 2.0d AT (177 л.с.),5.0,2007.0,WAGON_5_DOORS AUTOMATIC 2.0,автоматическая,2.0 LTR,177 N12,Причина продажи — покупка автомобиля выше клас...,267000.0,"['[{""name"":""Прочее"",""values"":[""Защита картера""...",задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,3 года и 11 месяцев,1315
2269,седан,BMW,белый,дизель,2013.0,M550d xDrive 3.0d AT (381 л.с.) 4WD,4.0,2013.0,SEDAN AUTOMATIC 3.0,автоматическая,3.0 LTR,381 N12,Есть цезарь + Webasto Машина в отличном состо...,149000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,10 месяцев,2269
2325,купе,BMW,синий,бензин,2013.0,420i xDrive 2.0 AT (184 л.с.) 4WD,2.0,2016.0,COUPE AUTOMATIC 2.0,автоматическая,2.0 LTR,184 N12,ДТП - нет!\n1. На паковочном месте зацепили за...,54000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,2 года и 2 месяца,2325


In [14]:
# Для корректного объединения датасетов переименуем vehicleConfiguration в vehicleTransmission в train
train.rename(columns={'vehicleConfiguration': 'vehicleTransmission'}, inplace=True)

# Для корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['Price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [15]:
data.sample(5)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id,sample,Price,Unnamed: 0
9880,Внедорожник 5 дв.,BMW,040001,бензин,2010.0,35i 3.0 AT (306 л.с.) 4WD,5.0,2010.0,NaN,AUTOMATIC,35i,306,Автомобиль куплен в Балт Авто Трейд-М 21.01.20...,162800.0,{'id': '0'},полный,LEFT,NaN,3,ORIGINAL,True,"{'year': 2011, 'month': 2}",NaN,1,1000000,10803.0
14493,Седан,BMW,0000CC,бензин,2001.0,320i 2.2 AT (170 л.с.),4.0,2002.0,NaN,AUTOMATIC,320i,170,"Продаётся отличный «Бумер», в достойном состоя...",280000.0,{'id': '0'},задний,LEFT,NaN,3,ORIGINAL,True,NaN,NaN,1,359999,79698.0
10827,Внедорожник 5 дв.,BMW,040001,бензин,2012.0,35i 3.0 AT (306 л.с.) 4WD,5.0,2013.0,NaN,AUTOMATIC,35i,306,Внимание! Данный автомобиль Вы можете приобрес...,66654.0,"{'id': '8494869', 'name': 'xDrive35i', 'availa...",полный,LEFT,NaN,2,ORIGINAL,True,NaN,NaN,1,1559000,11750.0
12506,Внедорожник 5 дв.,BMW,040001,дизель,2014.0,20d 2.0d AT (190 л.с.) 4WD,5.0,2017.0,NaN,AUTOMATIC,20d,190,Комплектация MSport. Состояние нового автомоби...,70000.0,"{'id': '21073109', 'name': 'xDrive20d M Sport ...",полный,LEFT,NaN,1,ORIGINAL,True,NaN,NaN,1,1950000,63850.0
12950,Внедорожник 5 дв.,BMW,200204,бензин,2012.0,18i 2.0 AT (150 л.с.),5.0,2014.0,NaN,AUTOMATIC,18i,150,NaN,70158.0,"{'id': '6937724', 'name': 'sDrive18i Локальная...",задний,LEFT,NaN,1,ORIGINAL,True,NaN,NaN,1,921000,67270.0


In [16]:
#
data.drop(columns=['name', 'vehicleConfiguration', 'engineDisplacement', 'description', 'Комплектация', 'Руль', 'Состояние', 'id', 'Unnamed: 0'], inplace=True,)

# Обработаем признак "Владение"
data['bodyType'] = data['bodyType'].apply(lambda x: str(x))
def bodyType(row):
    for body_type in ['внедорожник', 'хэтчбек', 'купе-хардтоп', 'седан', 'универсал', 
                      'родстер', 'кабриолет', 'компактвэн', 'лифтбек', 'купе']:
        if row.lower().startswith(body_type):
            return body_type
data['bodyType'] = data['bodyType'].apply(bodyType)
data['bodyType'] = data['bodyType'].apply(lambda x: 'купе' if x == 'купе-хардтоп' else x)

# Обработаем признак "color"
data['color'] = data['color'].map({'CACECB': 'серебристый', 'FAFBFB':'белый', 'EE1D19':'красный', '97948F':'серый', 
                                     '660099':'пурпурный', '040001':'чёрный','4A2197':'фиолетовый', 
                                     '200204':'коричневый','0000CC':'синий', '007F00':'зелёный', 'C49648':'бежевый',
                                     '22A0F8':'голубой','DEA522':'золотистый','FFD600': 'жёлтый', 'FF8649':'оранжевый',
                                     'FFC0CB':'розовый'})

# Обработаем признак "vehicleTransmission"
data['vehicleTransmission'] = data['vehicleTransmission'].map({'AUTOMATIC': 'автоматическая', 'MECHANICAL': 'механическая', 'ROBOT': 'роботизированная',})

# Обработаем признак "enginePower"
data['enginePower'] = data['enginePower'].apply(lambda x: x if isinstance(x, float) else int(str(x).split()[0]))

# Обработаем признак "Владельцы"
data['Владельцы'] = data['Владельцы'].apply(lambda x: x if isinstance(x, float) else int(str(x).split()[0]))

# Обработаем признак "ПТС"
data['ПТС'] = data['ПТС'].map({'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'})

# Mеняем названия колонок
data.columns = ['bodyType', 'color', 'fuelType', 'modelDate', 'numberOfDoors',
                'productionDate', 'vehicleTransmission', 'enginePower', 'mileage',
                'drive', 'numOwn', 'docs', 'durOwn', 'sample', 'Price']

# Удалаяем необработанные признаки
data.drop(['durOwn',], axis=1, inplace=True,)

ValueError: Length mismatch: Expected axis has 17 elements, new values have 15 elements

In [45]:
data.sample(5)

,bodyType,color,fuelType,modelDate,numberOfDoors,productionDate,vehicleTransmission,enginePower,mileage,drive,numOwn,docs,sample,Price
7493,седан,чёрный,бензин,1987.0,4.0,1990.0,механическая,150.0,250000.0,задний,3.0,Дубликат,1,215000
13553,хэтчбек,белый,бензин,2007.0,5.0,2011.0,автоматическая,136.0,105816.0,задний,2.0,Оригинал,1,595000
9789,внедорожник,серебристый,дизель,2013.0,5.0,2017.0,автоматическая,313.0,67600.0,полный,1.0,Оригинал,1,3450000
6624,седан,чёрный,дизель,2013.0,4.0,2014.0,автоматическая,258.0,115000.0,полный,2.0,Оригинал,1,1649000
347,внедорожник,NaN,дизель,2010.0,5.0,2013.0,NaN,184.0,79094.0,полный,1.0,NaN,0,0


In [46]:
data['numOwn'] = data['numOwn'].fillna(data['numOwn'].value_counts().idxmax())

for column in ['bodyType', 'color', 'fuelType', 'vehicleTransmission', 'drive', 'docs']:
    data[column] = data[column].astype('category').cat.codes  

data = pd.get_dummies(data, columns=['bodyType', 'color', 'fuelType', 'vehicleTransmission', 'drive', 'docs'])

for column in ['modelDate', 'numberOfDoors', 'productionDate', 'enginePower', 'mileage', 'numOwn']:
        data[column] = data[column].astype('int32')

In [47]:
#profile = ProfileReport(data, title="Pandas Profiling Report")
#profile.to_widgets()

In [48]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)
data = train_data

y = train_data['Price'].values
X = train_data.drop(['Price'], axis=1)
X_sub = test_data.drop(['Price'], axis=1)

# Train Split

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

# Fit&Submit AutoML

AutoMLRegressor

In [ ]:
model = AutoMLRegressor(X_train, y_train, X_test, random_state = RANDOM_SEED, verbose = 1)


In [ ]:
predict_test, predict_train = model.fit_predict(timeout = 8000, verbose = 1)

In [ ]:
model.stack_models_cfgs

In [ ]:
print('MAPE train:  ', mape(y_train, predict_train), '%')
print('MAPE test:  ', mape(y_test, predict_test), '%')

In [ ]:
#predict_submission = model.predict(X_sub)
#sample_submission['price'] = predict_submission
#sample_submission['price'] = sample_submission['price'].apply(lambda x: round(x/1000)*1000)
#sample_submission.to_csv(f'automl_ensemble.csv', index=False)

BestSingleModelRegressor

In [ ]:
model = BestSingleModelRegressor(X_train, y_train, X_test, random_state = RANDOM_SEED, verbose = 1)

In [ ]:
history = model.opt(timeout=8000, verbose=1)

In [ ]:
model.history_trials_dataframe.head(5)

In [ ]:
#model.plot_opt_history()

In [ ]:
predicts = model.predict()
predicts.head(5)

In [ ]:
print('MAPE train:  ', mape(y_train, predicts['predict_train'][0]), '%')
print('MAPE test:  ', mape(y_test, predicts['predict_test'][0]), '%')

In [ ]:
#predict_submission = model.predict(X_sub)
#sample_submission['price'] = predict_submission
#sample_submission['price'] = sample_submission['price'].apply(lambda x: round(x/1000)*1000)
#sample_submission.to_csv(f'automl_single.csv', index=False)

# Fit&Submit H2O

In [50]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,10 hours 4 mins
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_user_6vhchd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.708 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [51]:
df = h2o.H2OFrame(data)
y = 'Price'

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [52]:
aml = H2OAutoML(max_runtime_secs = 600, seed = RANDOM_SEED, project_name = "prj")
aml.train(y = y, training_frame = df)

AutoML progress: |█████
00:04:24.227: AutoML: XGBoost is not available; skipping it.
00:12:24.766: New models will be added to existing leaderboard prj@@Price (leaderboard frame=null) with already 21 models.
00:12:24.766: AutoML: XGBoost is not available; skipping it.
00:19:56.850: StackedEnsemble_BestOfFamily_AutoML_20201128_001224 [StackedEnsemble best (built using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
00:19:57.857: StackedEnsemble_AllModels_AutoML_20201128_001224 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
10:09:33.266: New models will be added to existing leade

In [53]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201128_000424,8.03942e+10,283539,8.03942e+10,135261,0.192037
StackedEnsemble_BestOfFamily_AutoML_20201128_000424,8.25544e+10,287323,8.25544e+10,134580,0.193543
XRT_1_AutoML_20201128_100933,8.46637e+10,290970,8.46637e+10,135443,0.191277
XRT_1_AutoML_20201128_001224,8.48358e+10,291266,8.48358e+10,135751,0.19125
DRF_1_AutoML_20201128_100933,8.70054e+10,294967,8.70054e+10,135846,0.193176
DRF_1_AutoML_20201128_001224,8.70854e+10,295102,8.70854e+10,136183,0.193249
XRT_1_AutoML_20201128_000424,8.87462e+10,297903,8.87462e+10,138779,0.194508
GBM_1_AutoML_20201128_001224,8.93836e+10,298971,8.93836e+10,152898,0.204518
GBM_1_AutoML_20201128_100933,8.93836e+10,298971,8.93836e+10,152898,0.204518
GBM_grid__1_AutoML_20201128_000424_model_1,8.97651e+10,299608,8.97651e+10,139916,0.205834


In [54]:
train = h2o.H2OFrame(X_train)
test = h2o.H2OFrame(X_test)
predict_train = aml.predict(train)
predict_test = aml.predict(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [55]:
y__test_pred = predict_test.as_data_frame()
y__test_pred = y__test_pred.apply(lambda x: round(x/1000)*1000)
y__test_pred = y__test_pred.to_numpy()
y__test_pred = y__test_pred.flatten().astype(int)

In [56]:
y_test

array([2820000,  640000, 1795000, ..., 1225000,   90000, 2490000],
      dtype=int64)

In [57]:
y__test_pred

array([3064000,  636000, 1733000, ..., 1292000,  125000, 2455000])

In [58]:
print('MAPE test:  ', mape(y_test, y__test_pred), '%')

MAPE test:   5.846 %
